# Image Renaming and Organization

This notebook processes images from a specified folder, renames them based on time progression, and saves the renamed images to an output directory. It handles grouping by well ID and ensures continued images are sequentially renamed with the correct time offset.


In [9]:
import os
import re
from pathlib import Path
from collections import defaultdict
from shutil import copy2

# --- CONFIGURATION ---
main_folder = "../data/g_m_20_1"     # adjust this to your folder name
continued_subfolder = "pokr"     # folder containing continued images
output_folder = "../data/g_m_20_1/renamed"        # output folder for renamed images
time_gap_hours = 118             # 04d14h00m + 8h = 118h

# --- HELPERS ---

def parse_time_hours(filename):
    match = re.search(r'(.*_)(\d{2})d(\d{2})h', filename)
    if not match:
        return None, None
    prefix = match.group(1)
    days = int(match.group(2))
    hours = int(match.group(3))
    total_hours = days * 24 + hours
    return prefix, total_hours

def format_hour(hour):
    days = hour // 24
    hours = hour % 24
    return f"{days:02}d{hours:02}h00m"

def get_images(folder):
    return sorted([
        f for f in os.listdir(folder)
        if f.lower().endswith((".jpg", ".png", ".jpeg"))
    ])

# --- MAIN ---

# Setup paths
main_path = Path(main_folder).resolve()
pokr_path = main_path / continued_subfolder
output_path = Path(output_folder).resolve()
output_path.mkdir(exist_ok=True)

# Group pokr images by well ID (like A10_1)
pokr_files_by_well = defaultdict(list)
for fname in get_images(pokr_path):
    match = re.search(r'([A-H]\d+_\d+)_', fname)
    if match:
        well_id = match.group(1)
        pokr_files_by_well[well_id].append(fname)

# Rename each continued image
for well_id, file_list in pokr_files_by_well.items():
    for i, fname in enumerate(sorted(file_list)):
        prefix, _ = parse_time_hours(fname)
        if not prefix:
            print(f"⚠️ Couldn't parse prefix from {fname}, skipping.")
            continue

        new_hour = time_gap_hours + i  # 1h spacing assumed
        new_time = format_hour(new_hour)
        ext = os.path.splitext(fname)[-1]
        new_name = prefix + new_time + ext

        src = pokr_path / fname
        dst = output_path / new_name
        copy2(src, dst)
        print(f"✅ {fname} → {new_name}")

print(f"\n✔️ All renamed images saved in: {output_path}")


✅ GBM_MSC_pokr_G10_1_00d00h00m.jpg → GBM_MSC_pokr_G10_1_04d22h00m.jpg
✅ GBM_MSC_pokr_G10_1_00d01h00m.jpg → GBM_MSC_pokr_G10_1_04d23h00m.jpg
✅ GBM_MSC_pokr_G10_1_00d02h00m.jpg → GBM_MSC_pokr_G10_1_05d00h00m.jpg
✅ GBM_MSC_pokr_G10_1_00d03h00m.jpg → GBM_MSC_pokr_G10_1_05d01h00m.jpg
✅ GBM_MSC_pokr_G10_1_00d04h00m.jpg → GBM_MSC_pokr_G10_1_05d02h00m.jpg
✅ GBM_MSC_pokr_G10_1_00d05h00m.jpg → GBM_MSC_pokr_G10_1_05d03h00m.jpg
✅ GBM_MSC_pokr_G10_1_00d06h00m.jpg → GBM_MSC_pokr_G10_1_05d04h00m.jpg
✅ GBM_MSC_pokr_G10_1_00d07h00m.jpg → GBM_MSC_pokr_G10_1_05d05h00m.jpg
✅ GBM_MSC_pokr_G10_1_00d08h00m.jpg → GBM_MSC_pokr_G10_1_05d06h00m.jpg
✅ GBM_MSC_pokr_G10_1_00d09h00m.jpg → GBM_MSC_pokr_G10_1_05d07h00m.jpg
✅ GBM_MSC_pokr_G10_1_00d10h00m.jpg → GBM_MSC_pokr_G10_1_05d08h00m.jpg
✅ GBM_MSC_pokr_G10_1_00d11h00m.jpg → GBM_MSC_pokr_G10_1_05d09h00m.jpg
✅ GBM_MSC_pokr_G10_1_00d12h00m.jpg → GBM_MSC_pokr_G10_1_05d10h00m.jpg
✅ GBM_MSC_pokr_G10_1_00d13h00m.jpg → GBM_MSC_pokr_G10_1_05d11h00m.jpg
✅ GBM_MSC_pokr_G10_1